In [1]:
import pandas as pd
import numpy as np
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas_datareader.data as web
import datetime as dt

pio.templates.default = 'presentation'

In [2]:
#Load stock data
company = "BTC-USD"
start = dt.date(2015, 1, 1)
end = dt.date.today()

df = web.DataReader(company, "yahoo", start, end)
df

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2015-01-01,320.434998,314.002991,320.434998,314.248993,8036550,314.248993
2015-01-02,315.838989,313.565002,314.079010,315.032013,7860650,315.032013
2015-01-03,315.149994,281.082001,314.846008,281.082001,33054400,281.082001
2015-01-04,287.230011,257.612000,281.145996,264.195007,55629100,264.195007
2015-01-05,278.341003,265.084015,265.084015,274.473999,43962800,274.473999
...,...,...,...,...,...,...
2022-04-04,46791.089844,45235.816406,46445.273438,46622.675781,32499785455,46622.675781
2022-04-05,47106.140625,45544.808594,46624.507812,45555.992188,29640604055,45555.992188
2022-04-06,45544.355469,43193.953125,45544.355469,43206.738281,39393395788,43206.738281


In [3]:
fig = px.line(df, y= "Close", title=f"{company} Price", labels={"Close":"Price $", "Date":"Year"})
fig.show()

In [4]:
def add_obv_and_obv_ema(df):
    '''Add On-Balance Volume (OBV) and 
    OBV Exponential Moving Average (EMA) for given dataframe.
    '''

    OBV = []
    OBV.append(0)

    #Loop through the data set (close price) from the second row (index 1) to the end of the data set
    for i in range(1, len(df.Close)):
        if df.Close[i] > df.Close[i-1]:
            OBV.append(OBV[-1] + df.Volume[i])
        elif df.Close[i] < df.Close[i-1]:
            OBV.append(OBV[-1] - df.Volume[i])
        else:
            OBV.append(OBV[-1])
    
    #Store the OBV and OBV Exponential Moving Average (EMA) into new columns
    df['OBV'] = OBV
    df['OBV_EMA'] = df['OBV'].ewm(span=20).mean()
    
    return df

In [5]:
df = web.DataReader(company, "yahoo", start, end)
df = add_obv_and_obv_ema(df)
df

,High,Low,Open,Close,Volume,Adj Close,OBV,OBV_EMA
Date,,,,,,,,
2015-01-01,320.434998,314.002991,320.434998,314.248993,8036550,314.248993,0,0.000000e+00
2015-01-02,315.838989,313.565002,314.079010,315.032013,7860650,315.032013,7860650,4.126841e+06
2015-01-03,315.149994,281.082001,314.846008,281.082001,33054400,281.082001,-25193750,-6.639504e+06
2015-01-04,287.230011,257.612000,281.145996,264.195007,55629100,264.195007,-80822850,-2.805509e+07
2015-01-05,278.341003,265.084015,265.084015,274.473999,43962800,274.473999,-36860050,-3.018493e+07
...,...,...,...,...,...,...,...,...
2022-04-04,46791.089844,45235.816406,46445.273438,46622.675781,32499785455,46622.675781,1954841053697,1.856078e+12
2022-04-05,47106.140625,45544.808594,46624.507812,45555.992188,29640604055,45555.992188,1925200449642,1.862662e+12
2022-04-06,45544.355469,43193.953125,45544.355469,43206.738281,39393395788,43206.738281,1885807053854,1.864866e+12


In [6]:
fig = px.line(df, x=df.index, y=["OBV", "OBV_EMA"], title=f"{company} OBV / OBV EMA", labels={"Date":"Year", "value":"Number of shares"})
fig.show()

In [7]:
#Create a function to signal when to buy and sell the stock
# If OBV > OBV_EMA Then Buy
# IF OBV < OBV_EMA Then Sell
# Else Do Nothing

def obv_indicator(df, col1, col2):
    ''' Add signal when to buy and sell depend on OBV and 
    OBV Exponential Moving Average (EMA)
    '''

    sigPriceBuy, sigPriceSell = [], []
    flag = -1
    #Loop through the length of the data set
    for i in range(0, len(df)):
        # If OBV > OBV_EMA Then Buy --> col1 => 'OBV' and col2 => 'OBV_EMA'
        if df[col1][i] > df[col2][i] and flag != 1:
            sigPriceBuy.append(df['Close'][i])
            sigPriceSell.append(np.nan)
            flag = 1
        # IF OBV < OBV_EMA Then Sell
        elif df[col1][i] < df[col2][i] and flag != 0:
            sigPriceSell.append(df['Close'][i])
            sigPriceBuy.append(np.nan)
            flag = 0
        else:
            sigPriceSell.append(np.nan)
            sigPriceBuy.append(np.nan)

    df['Buy_Signal_Price'] = sigPriceBuy
    df['Sell_Signal_Price'] = sigPriceSell      
    
    return df

In [8]:
#Create buy and sell columns
df = obv_indicator(df, 'OBV', 'OBV_EMA')
df

,High,Low,Open,Close,Volume,Adj Close,OBV,OBV_EMA,Buy_Signal_Price,Sell_Signal_Price
Date,,,,,,,,,,
2015-01-01,320.434998,314.002991,320.434998,314.248993,8036550,314.248993,0,0.000000e+00,NaN,NaN
2015-01-02,315.838989,313.565002,314.079010,315.032013,7860650,315.032013,7860650,4.126841e+06,315.032013,NaN
2015-01-03,315.149994,281.082001,314.846008,281.082001,33054400,281.082001,-25193750,-6.639504e+06,NaN,281.082001
2015-01-04,287.230011,257.612000,281.145996,264.195007,55629100,264.195007,-80822850,-2.805509e+07,NaN,NaN
2015-01-05,278.341003,265.084015,265.084015,274.473999,43962800,274.473999,-36860050,-3.018493e+07,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2022-04-04,46791.089844,45235.816406,46445.273438,46622.675781,32499785455,46622.675781,1954841053697,1.856078e+12,NaN,NaN
2022-04-05,47106.140625,45544.808594,46624.507812,45555.992188,29640604055,45555.992188,1925200449642,1.862662e+12,NaN,NaN
2022-04-06,45544.355469,43193.953125,45544.355469,43206.738281,39393395788,43206.738281,1885807053854,1.864866e+12,NaN,NaN


In [9]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=df.index, y=df.Close, name="Close Price", opacity=0.35),
    secondary_y=False,
)
fig.add_trace(
    go.Scatter(x=df.index, y=df["OBV"], name="OBV"),
    secondary_y=True,
)
fig.add_trace(
    go.Scatter(x=df.index, y=df["OBV_EMA"], name="OBV_EMA"),
    secondary_y=True,
)
fig.add_trace(
    go.Scatter(x=df.index, y=df['Buy_Signal_Price'], mode="markers", marker=dict(color="green", size=8, symbol="triangle-up", opacity=1), name="Buy Signal")
)
fig.add_trace(
    go.Scatter(x=df.index, y=df['Sell_Signal_Price'], mode="markers", marker=dict(color="red", size=8, symbol="triangle-down", opacity=1), name="Sell Signal")
)
# Add figure title
fig.update_layout(
    title_text=f"{company} Buy and Sell Signals"
)
# Set x-axis title
fig.update_xaxes(title_text="Date")
# Set y-axes titles
fig.update_yaxes(title_text="Close Price", secondary_y=False)
fig.show()

#### OBV Strategy Backtester

In [10]:
company = ["AAPL", "BTC-USD", "FB", "GOOG", "MSFT", "TSLA"]
start = dt.date(2015, 1, 1)
end = dt.date.today()
raw = web.DataReader(company, "yahoo", start, end)
raw.columns = pd.MultiIndex.from_tuples(raw.columns)
raw = raw.resample("B").mean().dropna()
raw.head()

Adj Close                                                \
                 AAPL     BTC-USD         FB        GOOG       MSFT   
Date                                                                  
2015-01-02  24.714510  286.769674  78.449997  523.373108  41.108841   
2015-01-05  24.018263  274.473999  77.190002  512.463013  40.730808   
2015-01-06  24.020521  286.188995  76.150002  500.585632  40.132992   
2015-01-07  24.357344  294.337006  76.150002  499.727997  40.642895   
2015-01-08  25.293211  283.348999  78.180000  501.303680  41.838520   

                           Close                                     ...  \
                 TSLA       AAPL     BTC-USD         FB        GOOG  ...   
Date                                                                 ...   
2015-01-02  43.862000  27.332500  286.769674  78.449997  523.373108  ...   
2015-01-05  42.018002  26.562500  274.473999  77.190002  512.463013  ...   
2015-01-06  42.256001  26.565001  286.188995  76.150002  500.585632  ...   
2015-01-07  42.189999  26.937500  294.337006  76.150002  499.727997  ...   
2015-01-08  42.124001  27.972500  283.348999  78.180000  501.303680  ...   

                 Open                                         Volume  \
                   FB        GOOG       MSFT       TSLA         AAPL   
Date                                                                   
2015-01-02  78.580002  527.561584  46.660000  44.574001  212818400.0   
2015-01-05  77.980003  521.827332  46.369999  42.910000  257142000.0   
2015-01-06  77.230003  513.589966  46.380001  42.012001  263188400.0   
2015-01-07  76.760002  505.611847  45.980000  42.669998  160423600.0   
2015-01-08  76.739998  496.626526  46.750000  42.562000  237458000.0   

                                                                         
                 BTC-USD          FB       GOOG        MSFT        TSLA  
Date                                                                     
2015-01-02  3.218138e+07  18177500.0  1447563.0  27913900.0  23822000.0  
2015-01-05  4.396280e+07  26452200.0  2059840.0  39673900.0  26842500.0  
2015-01-06  2.324570e+07  27399300.0  2899940.0  36447900.0  31309500.0  
2015-01-07  2.486680e+07  22045300.0  2065054.0  29114100.0  14842000.0  
2015-01-08  1.998250e+07  23961000.0  3353582.0  29645200.0  17212500.0  

[5 rows x 36 columns]

In [11]:
raw = raw[['Close', 'Volume']]
raw = raw.swaplevel(axis = 1).sort_index(axis = 1)
raw.head()

AAPL                  BTC-USD                       FB  \
                Close       Volume       Close        Volume      Close   
Date                                                                      
2015-01-02  27.332500  212818400.0  286.769674  3.218138e+07  78.449997   
2015-01-05  26.562500  257142000.0  274.473999  4.396280e+07  77.190002   
2015-01-06  26.565001  263188400.0  286.188995  2.324570e+07  76.150002   
2015-01-07  26.937500  160423600.0  294.337006  2.486680e+07  76.150002   
2015-01-08  27.972500  237458000.0  283.348999  1.998250e+07  78.180000   

                              GOOG                  MSFT              \
                Volume       Close     Volume      Close      Volume   
Date                                                                   
2015-01-02  18177500.0  523.373108  1447563.0  46.759998  27913900.0   
2015-01-05  26452200.0  512.463013  2059840.0  46.330002  39673900.0   
2015-01-06  27399300.0  500.585632  2899940.0  45.650002  36447900.0   
2015-01-07  22045300.0  499.727997  2065054.0  46.230000  29114100.0   
2015-01-08  23961000.0  501.303680  3353582.0  47.590000  29645200.0   

                 TSLA              
                Close      Volume  
Date                               
2015-01-02  43.862000  23822000.0  
2015-01-05  42.018002  26842500.0  
2015-01-06  42.256001  31309500.0  
2015-01-07  42.189999  14842000.0  
2015-01-08  42.124001  17212500.0

In [12]:
symbol = "BTC-USC"
raw = raw["BTC-USD"]
raw = raw.loc[start:end]
raw.head()

,Close,Volume
Date,,
2015-01-02,286.769674,3.218138e+07
2015-01-05,274.473999,4.396280e+07
2015-01-06,286.188995,2.324570e+07
2015-01-07,294.337006,2.486680e+07
2015-01-08,283.348999,1.998250e+07


In [13]:
raw["returns"] = np.log(raw.Close / raw.Close.shift(1))
raw.head()

,Close,Volume,returns
Date,,,
2015-01-02,286.769674,3.218138e+07,NaN
2015-01-05,274.473999,4.396280e+07,-0.043823
2015-01-06,286.188995,2.324570e+07,0.041796
2015-01-07,294.337006,2.486680e+07,0.028073
2015-01-08,283.348999,1.998250e+07,-0.038046
